## 第07讲：图像切分不合理？文本图像矫正和版面区域检测保障信息完整且不冗余

## 一、问题背景


在上一讲中，我们介绍了 PaddleOCR 强大的图像识别能力和表格文字抽取能力。

但有时我们并不想使用完整的 OCR 流程，而是希望利用其中的某些模块来增强自己的系统。

比如：
> “我已经有自己偏好的 OCR 工具了，但对 PaddleOCR 的图像预处理能力很感兴趣。”

PaddleOCR 并不仅仅是一个整体 OCR 工具包，它还提供了多个可独立调用的功能模块，例如：

- ✅ 文本图像矫正（Text Image Unwarping）
- ✅ 版面区域检测（Layout Detection）

这两个模块可以有效提升图像质量、定位关键区域、避免冗余信息干扰，是构建高质量 OCR 系统的重要组成部分。


## 二、版面区域检测模块详解

### 2.1 什么是版面区域检测？

[版面区域检测](https://paddlepaddle.github.io/PaddleX/latest/module_usage/tutorials/ocr_modules/layout_detection.html#_2)
的核心任务是对输入的文档图像进行内容解析和区域划分。通过识别图像中的不同元素（如文字、图表、图像、公式、段落、摘要、参考文献等），将其归类为预定义的类别，并确定这些区域在文档中的位置。

这一步对于后续的 OCR 识别、信息抽取、结构化输出等环节至关重要，能够有效提升整体识别的准确性和逻辑性。



### 2.2 支持的模型与类别

PaddleOCR 提供了多种版面检测模型，支持多达 **23个常见类别**，包括但不限于：

- 文档标题、段落标题  
- 文本、页码、摘要  
- 目录、参考文献、脚注  
- 页眉、页脚、算法  
- 公式、公式编号  
- 图像、表格、图和表标题  
- 印章、图表、侧栏文本  

#### 支持模型列表（部分）：

| 模型 | mAP(0.5) (%) | GPU推理耗时 (ms) | CPU推理耗时 (ms) | 模型大小 (MB) | 描述 |
|------|---------------|------------------|------------------|---------------|------|
| PP-DocLayout-L | 90.4 | 34.62 / 10.39 | 510.57 / - | 123.76 | 高精度版面区域定位模型 |
| PP-DocLayout-M | 75.2 | 13.33 / 4.87 | 44.07 / 44.07 | 22.58 | 精度效率平衡模型 |
| PP-DocLayout-S | 70.9 | 8.30 / 2.38 | 10.06 / 9.93 | 4.83 | 高效率模型 |

完整模型列表详见官方文档。



### 2.3 示例代码演示


In [1]:
# 能正确识别的图像

from paddleocr import LayoutDetection

model = LayoutDetection(model_name="PP-DocLayout_plus-L")
output = model.predict("./pics/image_good.png", batch_size=1, layout_nms=True)
for res in output:
    res.print()
    res.save_to_img(save_path="./output07/")
    res.save_to_json(save_path="./output07/res.json")

/opt/miniconda3/envs/rag50/lib/python3.12/site-packages/paddle/utils/cpp_extension/extension_utils.py:715: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
MKL-DNN is not available. Using `paddle` instead.
Using official model (PP-DocLayout_plus-L), the model files will be automatically downloaded and saved in /Users/wilson/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

{'res': {'input_path': './pics/image_good.png', 'page_index': None, 'boxes': [{'cls_id': 2, 'label': 'text', 'score': 0.9845171570777893, 'coordinate': [np.float32(90.02504), np.float32(149.24373), np.float32(541.56226), np.float32(323.24805)]}, {'cls_id': 1, 'label': 'image', 'score': 0.9734574556350708, 'coordinate': [np.float32(106.04124), np.float32(329.02234), np.float32(543.95496), np.float32(527.4854)]}, {'cls_id': 2, 'label': 'text', 'score': 0.972456157207489, 'coordinate': [np.float32(614.44977), np.float32(338.04285), np.float32(1020.8649), np.float32(408.39597)]}, {'cls_id': 2, 'label': 'text', 'score': 0.9719167947769165, 'coordinate': [np.float32(598.0112), np.float32(138.84532), np.float32(1019.111), np.float32(228.49037)]}, {'cls_id': 7, 'label': 'formula', 'score': 0.9527032971382141, 'coordinate': [np.float32(623.0048), np.float32(81.488174), np.float32(1005.11096), np.float32(122.09485)]}, {'cls_id': 7, 'label': 'formula', 'score': 0.9476105570793152, 'coordinate': [

### 这段代码的主要功能是？

该代码调用的是 PaddleOCR 提供的 **版面区域检测模块**，用于对输入文档图像进行布局分析。具体功能包括：

- 使用指定模型（如 `PP-DocLayout_plus-L`）对图像进行推理预测；
- 输出每个检测到的区域及其类别标签、置信度和边界框坐标；
- 将检测结果可视化为图像文件；
- 将检测结果保存为 JSON 格式，便于后续处理与集成。

### 它为我们解决的问题是？

在实际 OCR 应用中，原始图像可能存在多个不同类型的内容区域（如表格、文本、图像、公式等）。如果直接对整张图进行识别，容易导致：

- 信息混杂、识别混乱；
- 多余区域干扰主内容识别；
- 后续结构化提取困难。

而通过版面区域检测模块，我们可以：

✅ 精准定位图像中的各个内容区域  
✅ 对不同类型的区域进行分类管理（如表格、文本、图像等）  
✅ 避免冗余信息干扰，提高 OCR 的准确率和结构化输出效率

上面我们介绍了 PaddleOCR 的版面区域检测模块，它可以帮助我们精准地识别结构清晰、排版良好的文档图像。

但是，现实中的图像并不总是“规规矩矩”的。比如：

- 文档被拍摄时发生了**倾斜**；
- 纸张有**弯曲、褶皱**；
- 存在**透视变形**（比如从侧面拍照）；
- 图像**模糊**或有**扭曲现象**。

这些情况都会严重影响 OCR 的识别效果，甚至导致完全无法识别。

这时，我们就需要引入 PaddleOCR 中的另一个强大模块 —— **文本图像矫正模块**。

---

## 3.1 文本图像矫正模块

文本图像矫正如其名，它的主要目的是对图像进行**几何变换**，以纠正其中文档的**扭曲、倾斜、透视变形**等问题，使得图像更接近“标准”文档的样子，从而为后续的文本识别提供更高质量的输入。

### 实战演示：用代码体验文本图像矫正

下面我用一段代码来演示一下这个模块的使用方式：


In [6]:
# 不能正确识别的图像

from paddleocr import LayoutDetection

model = LayoutDetection(model_name="PP-DocLayout_plus-L")
output = model.predict("./pics/image.jpg", batch_size=1, layout_nms=True)
for res in output:
    res.print()
    res.save_to_img(save_path="./output07/")

MKL-DNN is not available. Using `paddle` instead.
Using official model (PP-DocLayout_plus-L), the model files will be automatically downloaded and saved in /Users/wilson/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

{'res': {'input_path': './pics/image.jpg', 'page_index': None, 'boxes': [{'cls_id': 2, 'label': 'text', 'score': 0.7575176358222961, 'coordinate': [np.float32(1285.9551), np.float32(1711.5494), np.float32(1714.0671), np.float32(2082.0159)]}, {'cls_id': 2, 'label': 'text', 'score': 0.6536715626716614, 'coordinate': [np.float32(855.67084), np.float32(1611.6976), np.float32(1208.881), np.float32(2212.0989)]}, {'cls_id': 2, 'label': 'text', 'score': 0.6392190456390381, 'coordinate': [np.float32(1814.1279), np.float32(1684.3706), np.float32(2202.0146), np.float32(2141.768)]}, {'cls_id': 2, 'label': 'text', 'score': 0.5184097290039062, 'coordinate': [np.float32(1303.5271), np.float32(2241.2612), np.float32(2005.0392), np.float32(2344.6514)]}, {'cls_id': 0, 'label': 'paragraph_title', 'score': 0.5072970986366272, 'coordinate': [np.float32(1331.5743), np.float32(1648.6868), np.float32(1722.4619), np.float32(1712.2705)]}]}}


In [7]:
from paddleocr import TextImageUnwarping
model = TextImageUnwarping(model_name="UVDoc")
output = model.predict("./pics/image.jpg", batch_size=1)
for res in output:
    res.print()
    res.save_to_img(save_path="./pics/")

MKL-DNN is not available. Using `paddle` instead.
Using official model (UVDoc), the model files will be automatically downloaded and saved in /Users/wilson/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

{'res': {'input_path': './pics/image.jpg', 'page_index': None, 'doctr_img': '...'}}


### 这段代码做了什么？

使用 TextImageUnwarping 类加载了名为 UVDoc 的文本图像矫正模型；

对本地文件 doc_test.jpg 进行图像矫正推理；

输出矫正后的图像结果，并保存为图片和 JSON 文件，便于后续分析和调用。

运行后你会看到，原本倾斜或扭曲 的文档图像已经被自动“拉直”，变得更适合 OCR 识别。

In [8]:
# 能正确识别的图像

from paddleocr import LayoutDetection

model = LayoutDetection(model_name="PP-DocLayout_plus-L")
output = model.predict("./pics/image_res.jpg", batch_size=1, layout_nms=True)
for res in output:
    res.print()
    res.save_to_img(save_path="./output07-2/")
    res.save_to_json(save_path="./output07-2/res.json")

MKL-DNN is not available. Using `paddle` instead.
Using official model (PP-DocLayout_plus-L), the model files will be automatically downloaded and saved in /Users/wilson/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

{'res': {'input_path': './pics/image_res.jpg', 'page_index': None, 'boxes': [{'cls_id': 2, 'label': 'text', 'score': 0.8861143589019775, 'coordinate': [np.float32(996.63605), np.float32(1533.3918), np.float32(1581.1954), np.float32(1925.9213)]}, {'cls_id': 2, 'label': 'text', 'score': 0.7849099636077881, 'coordinate': [np.float32(158.78006), np.float32(1498.5908), np.float32(881.00464), np.float32(2186.8235)]}, {'cls_id': 2, 'label': 'text', 'score': 0.771456778049469, 'coordinate': [np.float32(1715.4049), np.float32(1525.8024), np.float32(2257.6177), np.float32(1989.1323)]}, {'cls_id': 2, 'label': 'text', 'score': 0.6567004919052124, 'coordinate': [np.float32(1019.5312), np.float32(2121.3577), np.float32(1944.7799), np.float32(2233.2126)]}, {'cls_id': 0, 'label': 'paragraph_title', 'score': 0.6213204860687256, 'coordinate': [np.float32(1091.3735), np.float32(1457.0441), np.float32(1592.3569), np.float32(1521.407)]}, {'cls_id': 0, 'label': 'paragraph_title', 'score': 0.5970951914787292

# 针对表格内容，我们有另一套完整的解决方案！

在处理包含表格的文档图像时，仅仅依靠 OCR 是不够的。为了更高效、准确地提取表格信息，PaddleOCR 提供了一套完整的表格识别模块体系。

这套方案包括三个核心模块：

- 表格分类模块  
- 表格单元格检测模块  
- 表格结构识别模块  

它们各自解决的问题如下表所示：

| 模块名称             | 主要功能                                     | 解决问题示例                             |
|----------------------|----------------------------------------------|------------------------------------------|
| 表格分类模块         | 对表格图像进行分类（如有线表、无线表等）     | 为后续识别流程提供先验信息               |
| 表格单元格检测模块   | 定位并标记每个单元格的边界框                 | 精准划分单元格区域，便于内容提取         |
| 表格结构识别模块     | 将表格图像转换为可编辑的 HTML 结构           | 实现表格结构还原和语义理解               |

接下来我将逐一为你演示这三个模块的功能和使用方式。

## 4.1 表格分类模块：为表格识别提供“第一印象”

表格分类模块是计算机视觉系统中的关键组成部分，负责对输入的表格图像进行分类。

该模块的性能直接影响到整个表格识别过程的准确性和效率。

它通常接收表格图像作为输入，然后通过深度学习算法，根据图像的特性和内容，将其分类到预定义的类别中，例如：

- 有线表（线条清晰可见）
- 无线表（无明显边框）

分类结果将作为后续模块的重要参考，帮助系统选择合适的识别策略。

示例输出：

In [9]:
from paddleocr import TableClassification
model = TableClassification(model_name="PP-LCNet_x1_0_table_cls")
output = model.predict("./pics/table_recognition.png", batch_size=1)
for res in output:
    res.print(json_format=False)
    res.save_all("./output07-3")

MKL-DNN is not available. Using `paddle` instead.
Using official model (PP-LCNet_x1_0_table_cls), the model files will be automatically downloaded and saved in /Users/wilson/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

{'res': {'input_path': './pics/table_recognition.png', 'page_index': None, 'class_ids': array([0, 1], shape=(2,), dtype=int32), 'scores': array([0.84421, 0.15579], shape=(2,), dtype=float32), 'label_names': ['wired_table', 'wireless_table']}}


## 4.2 表格单元格检测模块：精准定位每一个单元格

表格单元格检测模块是表格识别任务的关键组成部分。

它的主要职责是在表格图像中定位和标记每个单元格区域 。

模块通常会输出各个单元格区域的边界框（Bounding Boxes），这些边界框将作为输入传递给后续流程进行内容识别和结构重建。

输出示例：

In [10]:
from paddleocr import TableCellsDetection
model = TableCellsDetection(model_name="RT-DETR-L_wired_table_cell_det")
output = model.predict("./pics/table_recognition.png", threshold=0.3, batch_size=1)
for res in output:
    res.print(json_format=False)
    res.save_to_img("./output07-4")

MKL-DNN is not available. Using `paddle` instead.
Using official model (RT-DETR-L_wired_table_cell_det), the model files will be automatically downloaded and saved in /Users/wilson/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

{'res': {'input_path': './pics/table_recognition.png', 'page_index': None, 'boxes': [{'cls_id': 0, 'label': 'cell', 'score': 0.973805844783783, 'coordinate': [np.float32(2.1718848), 0, np.float32(546.6909), np.float32(30.749983)]}, {'cls_id': 0, 'label': 'cell', 'score': 0.9714836478233337, 'coordinate': [np.float32(212.63258), np.float32(30.765705), np.float32(403.77368), np.float32(64.7576)]}, {'cls_id': 0, 'label': 'cell', 'score': 0.971441388130188, 'coordinate': [np.float32(212.49472), np.float32(64.74863), np.float32(403.75485), np.float32(95.76425)]}, {'cls_id': 0, 'label': 'cell', 'score': 0.9697222113609314, 'coordinate': [np.float32(403.74326), np.float32(30.796143), np.float32(547.1804), np.float32(64.749825)]}, {'cls_id': 0, 'label': 'cell', 'score': 0.9692213535308838, 'coordinate': [np.float32(403.74915), np.float32(64.73799), np.float32(547.0551), np.float32(95.78832)]}, {'cls_id': 0, 'label': 'cell', 'score': 0.9691973924636841, 'coordinate': [np.float32(109.67606), np.

## 4.3 表格结构识别模块：从图像到可编辑表格

表格结构识别模块是表格识别系统中的“最终呈现层”。

它的目标是将不可编辑的表格图像转换为可编辑的结构化形式 ，如 HTML 表格。

通过对表格的行、列和单元格位置进行识别，该模块可以生成标准的 HTML 代码，实现表格的完整结构还原。

In [11]:
from paddleocr import TableStructureRecognition
model = TableStructureRecognition(model_name="SLANet")
output = model.predict(input="./pics/table_recognition.png", batch_size=1)
for res in output:
    res.print(json_format=False)
    res.save_all("./output07-5/")


MKL-DNN is not available. Using `paddle` instead.
Using official model (SLANet), the model files will be automatically downloaded and saved in /Users/wilson/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

{'res': {'input_path': './pics/table_recognition.png', 'page_index': None, 'bbox': [[42, 2, 390, 2, 388, 27, 40, 26], [11, 35, 89, 35, 87, 63, 11, 63], [113, 34, 192, 34, 186, 64, 109, 64], [219, 33, 399, 33, 393, 62, 212, 62], [413, 33, 544, 33, 544, 64, 407, 64], [12, 67, 98, 68, 96, 93, 12, 93], [115, 66, 205, 66, 200, 91, 111, 91], [234, 65, 390, 65, 385, 92, 227, 92], [414, 66, 537, 67, 537, 95, 409, 95], [7, 97, 106, 97, 104, 128, 7, 128], [113, 96, 206, 95, 201, 127, 109, 127], [236, 96, 386, 96, 381, 128, 230, 128], [413, 96, 534, 95, 533, 127, 408, 127]], 'structure': ['<html>', '<body>', '<table>', '<tr>', '<td', ' colspan="4"', '>', '</td>', '</tr>', '<tr>', '<td></td>', '<td></td>', '<td></td>', '<td></td>', '</tr>', '<tr>', '<td></td>', '<td></td>', '<td></td>', '<td></td>', '</tr>', '<tr>', '<td></td>', '<td></td>', '<td></td>', '<td></td>', '</tr>', '</table>', '</body>', '</html>'], 'structure_score': np.float32(0.99948007)}}


## 5 本讲小结
在本讲中，我们深入探讨了 OCR 处理中文档图像常见的“图像切分不合理”问题，并介绍了 PaddleOCR 提供的两个关键预处理模块：

- 文本图像矫正模块（Text Image Unwarping） ：用于纠正图像中的文档扭曲、倾斜、透视变形等问题，使图像更接近标准文档形式，从而提升识别准确率。
- 版面区域检测模块（Layout Detection） ：对文档图像进行内容解析和区域划分，精准识别出如文字、表格、图像、公式等多种元素，并分类标注其位置，保障信息完整且不冗余。

这些模块都可以独立集成到你自己的 OCR 流程中 ，灵活应对各种复杂场景下的图像质量问题。

此外，我们也了解到，在面对表格类文档时，PaddleOCR 还提供了一整套完整的识别流程：
表格分类模块：识别表格类型
表格单元格检测模块：定位每个单元格边界
表格结构识别模块：输出 HTML 形式的结构化表格
通过这些模块的协同工作，我们可以实现从原始图像到结构化信息的完整转换，显著提升 OCR 系统在复杂文档场景下的鲁棒性和实用性。

